In [194]:
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression
from sklearn.metrics import r2_score
import numpy as np
import pandas as pd
from scipy.stats import linregress
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
plt.rcParams['font.sans-serif']=['SimHei'] 
plt.rcParams['axes.unicode_minus']=False 


### 数据准备

In [184]:
inr=pd.read_csv("D:/HuaweiMoveData/Users/zxh/Desktop/风险高公司data.csv")
inry=pd.read_csv("D:/HuaweiMoveData/Users/zxh/Desktop/风险低公司data.csv")


### 数据预处理

In [185]:
inr.set_index([inr['asharedebt_stat_symbol'],inr['asharedebt_stat_stat_date']],inplace=True)
inry.set_index([inry['asharedebt_stat_symbol'],inry['asharedebt_stat_stat_date']],inplace=True)
inr.drop(columns=['Unnamed: 0','asharedebt_stat_symbol','asharedebt_stat_stat_date'],inplace=True)
inry.drop(columns=['Unnamed: 0','asharedebt_stat_symbol','asharedebt_stat_stat_date'],inplace=True)

### PCA降维

In [186]:
df=pd.concat([inr,inry])
df = df.fillna(method='ffill')
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)
pca = PCA(n_components=15)  
principalComponents = pca.fit_transform(df_scaled)
explained_variance_ratio = pca.explained_variance_ratio_
cumulative_explained_variance = np.cumsum(explained_variance_ratio)
principalDf = pd.DataFrame(data=principalComponents,
                        columns=['pc1', 'pc2', 'pc3', 'pc4', 'pc5', 'pc6', 'pc7', 'pc8', 'pc9', 'pc10', 'pc11', 'pc12', 'pc13', 'pc14', 'pc15'])
df.reset_index(inplace=True)
principalDf.set_index([df['asharedebt_stat_symbol'],df['asharedebt_stat_stat_date']],inplace=True)


C:\Users\zxh\AppData\Local\Temp\ipykernel_13648\850773329.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill')


### 公司分类

In [187]:
def check_conditions(df):
    # 检查 cashflow_net_cash_flows_from_opt_act/income_operating_income 是否有负值
    if df['cashflow_net_cash_flows_from_opt_act/income_operating_income'].lt(0).any() and df['profit_stat_roa'].lt(0).any():
            # 检查是否有任意一列连续出现3个负数
        if (df['cashflow_net_cash_flows_from_opt_act/income_operating_income'].lt(0).rolling(3).sum().max() >= 3) or \
            (df['profit_stat_roa'].lt(0).rolling(3).sum().max() >= 3):
            return 3
        else:
            return 2
    else:
        return 1

sgp=pd.concat([inr,inry])
safe=[]
doubt=[]
risk=[]
for i in list(set(df['asharedebt_stat_symbol'].values)):
    df=sgp.loc[i,['cashflow_net_cash_flows_from_opt_act/income_operating_income','profit_stat_roa']]
    result = check_conditions(df)
    if result==1:
        safe.append(i)
    elif result==2:
        doubt.append(i)
    else:
        risk.append(i)


### 阈值确定

+ 小于down为safe，大于up为risk 

In [188]:
doubt_data=principalDf.loc[doubt]
doubt_data['sum']=doubt_data.values@explained_variance_ratio
confidence_level = 0.95 
ci = sm.stats.DescrStatsW(doubt_data['sum']).tconfint_mean(alpha=(1 - confidence_level))
down=ci[0]
up=ci[1]

### kalman滤波预测

In [189]:
# Q为这一轮的心里的预估误差
Q = 0.1
# R为下一轮的测量误差
R = 1
# Accumulated_Error为上一轮的估计误差，具体呈现为所有误差的累计
Accumulated_Error = 1
# 初始旧值
kalman_adc_old = 0
SCOPE = 50
def kalman(ADC_Value):
    global kalman_adc_old
    global Accumulated_Error
    # 新的值相比旧的值差太大时进行跟踪
    if (abs(ADC_Value-kalman_adc_old)/SCOPE > 0.25):
        Old_Input = ADC_Value*0.382 + kalman_adc_old*0.618
    else:
        Old_Input = kalman_adc_old
    # 上一轮的 总误差=累计误差^2+预估误差^2
    Old_Error_All = (Accumulated_Error**2 + Q**2)**(1/2)
    # R为这一轮的预估误差
    # H为利用均方差计算出来的双方的相信度
    H = Old_Error_All**2/(Old_Error_All**2 + R**2)
    # 旧值 + 1.00001/(1.00001+0.1) * (新值-旧值)
    kalman_adc = Old_Input + H * (ADC_Value - Old_Input)
    # 计算新的累计误差
    Accumulated_Error = ((1 - H)*Old_Error_All**2)**(1/2)
    # 新值变为旧值
    kalman_adc_old = kalman_adc
    return kalman_adc

### 图像显示

In [190]:
stock_data=principalDf
stock_data['sum']=stock_data.values@explained_variance_ratio
kal_data={}
for s in safe+doubt+risk:
    adc_values=stock_data.loc[s]['sum']
    kal_array=[]
    for i in range(len(adc_values)):
        kal_array.append(kalman(adc_values[i]))
    kal_data[s]=kal_array

C:\Users\zxh\AppData\Local\Temp\ipykernel_13648\3810466090.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  kal_array.append(kalman(adc_values[i]))


#### 安全企业

In [ ]:
for i in safe:
    real_data = stock_data.loc[i]['sum'].values
    predict_data = kal_data[i]
    plt.figure(figsize=(10, 5))
    plt.title(i)
    plt.plot(real_data, label='真实值', color='blue')
    plt.plot(predict_data, label='预测值', color='red', linestyle='--')
    plt.axhline(y=down, color='green', linestyle=':', label='安全线')
    plt.axhline(y=up, color='red', linestyle=':', label='风险线')
    plt.legend()
    plt.grid(True)
    plt.show()

#### 重度风险企业

In [ ]:
for i in risk[0:6]:
    real_data = stock_data.loc[i]['sum'].values
    predict_data = kal_data[i]
    plt.figure(figsize=(10, 5))
    plt.title(i)
    plt.plot(real_data, label='真实值', color='blue')
    plt.plot(predict_data, label='预测值', color='red', linestyle='--')
    plt.axhline(y=down, color='green', linestyle=':', label='安全线')
    plt.axhline(y=up, color='red', linestyle=':', label='风险线')
    plt.legend()
    plt.grid(True)
    plt.show()

#### 轻度风险企业

In [ ]:
for i in doubt[0:6]:
    real_data = stock_data.loc[i]['sum'].values
    predict_data = kal_data[i]
    plt.figure(figsize=(10, 5))
    plt.title(i)
    plt.plot(real_data, label='真实值', color='blue')
    plt.plot(predict_data, label='预测值', color='red', linestyle='--')
    plt.axhline(y=down, color='green', linestyle=':', label='安全线')
    plt.axhline(y=up, color='red', linestyle=':', label='风险线')
    plt.legend()
    plt.grid(True)
    plt.show()